In [25]:
# Biblioteki

import pandas as pd 
import numpy as np
import pymysql
import mysql.connector
import datetime as dt
import os
import glob

print("Script is starting, the current time is", pd.to_datetime("today"))

# Variables

influ_window_days = 60
cc_window_days = 60
now = pd.to_datetime("today")
consumption_time = 45
excl = 60

# Import Vicidial

vicidial = pd.read_excel(r"C:\Users\stefa\Desktop\excele\vicidial\vicidial.xlsx")
vicidial

replacements = {
  r'.*No Answer AutoDial*': 'Nieodebrane',
  r'.*Call Back*': 'Nieodebrane',
  r'.*Poczta Głosowa*': 'Nieodebrane',
  r'.*Zamówienie- nowe*': 'Sukces'
}

vicidial['status_grup'] = vicidial.status_name.replace(replacements, regex=True)
vicidial.loc[vicidial.status_grup.str.contains('\.'), 'status_grup'] = 'Odebrane'
m = (vicidial['status_grup'] == 'Nieodebrane') | (vicidial['status_grup'] == 'Sukces')
vicidial['status_grup'] = np.where(m, vicidial['status_grup'], 'Odebrane')
vicidial

def sukces_01(row):
  if row['status_grup'] == "Sukces":
    return 6
  else:
    return 0
vicidial['sukces_01'] = vicidial.apply(sukces_01, axis=1)

def odebrane_01(row):
  if row['status_grup'] == "Odebrane":
    return 2
  else:
    return 0
vicidial['odebrane_01'] = vicidial.apply(odebrane_01, axis=1)

def Nieodebrane_01(row):
  if row['status_grup'] == "Nieodebrane":
    return -2
  else:
    return 0
vicidial['Nieodebrane_01'] = vicidial.apply(Nieodebrane_01, axis=1)
vicidial['stat_sum'] = vicidial['Nieodebrane_01'] + vicidial['odebrane_01'] + vicidial['sukces_01']

score_historia = vicidial.groupby('phone_number_dialed')['stat_sum'].sum().reset_index()
score_historia = score_historia.set_index('phone_number_dialed')

telefony_grup_ost = vicidial.groupby('phone_number_dialed')['call_date'].max().reset_index()
telefony_grup_ost = telefony_grup_ost.rename(columns={"call_date":"data_ost"})

vicidial = vicidial.merge(telefony_grup_ost, on='phone_number_dialed', how= 'left')

def status_ost(row):
  if row['call_date'] == row['data_ost']:
    return 1
  else:
    return 0
vicidial['status_ost'] = vicidial.apply(status_ost, axis=1)
vicidial.loc[(vicidial["status_ost"] == 1) & (vicidial["sukces_01"] == 6),"status_ost_waga"] = 6
vicidial.loc[(vicidial["status_ost"] == 1) & (vicidial["odebrane_01"] == 2),"status_ost_waga"] = 2
vicidial.loc[(vicidial["status_ost"] == 1) & (vicidial["Nieodebrane_01"] == -2),"status_ost_waga"] = -2
vicidial.loc[(vicidial["status_ost"] == 0),"status_ost_waga"] = 0
vicidial['stat_sum_ost'] = vicidial['stat_sum'] + vicidial['status_ost_waga']

score_ostatnie_zam = vicidial.groupby('phone_number_dialed')['stat_sum_ost'].sum().reset_index()
score_ostatnie_zam = score_ostatnie_zam.set_index('phone_number_dialed')

score_pol = score_historia.merge(score_ostatnie_zam, on= 'phone_number_dialed', how= 'left')
score_pol
# Połączenie SQL

con = mysql.connector.connect(
  host="prod-replica.ceufrkrz6jt9.eu-central-1.rds.amazonaws.com",
  user="mdziekonski",
  password="ErhoUoD@46%EokESiR@1xq"
)

print(con)
mycursor = con.cursor()

# SQL QUERIES
# Clients Query

clients_query =  """select so.customer_email, so.sales_channel, so.entity_id, so.created_at,
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) AS 'customer_max',
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) - so.created_at as max_diff,
                  datediff(MAX(so.created_at) OVER(PARTITION BY so.customer_email), so.created_at) as days_diff,
                  base_subtotal, products
                  from magento.sales_order so
                  where so.state != 'canceled'
                  and so.status != 'canceled'
                  and so.is_valid = 1
                  and so.store_id = 1
                  and so.customer_email is not null
                  and so.customer_email not like '%healthlabs%'
                  and so.customer_email not like '%adequate%'
                  and so.coupon_code NOT IN ('Motywacja15', 'Dietoterapia10', '4US50', '4FR33',
                  'PGFRIENDS35', '4THEM30', 'LORENC', 'LORENC10', 'LORENC25')
UNION ALL
select so.customer_email, so.sales_channel, so.entity_id, so.created_at,
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) AS 'customer_max',
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) - so.created_at as max_diff,
                  datediff(MAX(so.created_at) OVER(PARTITION BY so.customer_email), so.created_at) as days_diff,
                  base_subtotal, products
                  from magento.sales_order so
                  where so.state != 'canceled'
                  and so.status != 'canceled'
                  and so.is_valid = 1
                  and so.store_id = 1
                  and so.customer_email is not null
                  and so.customer_email not like '%healthlabs%'
                  and so.customer_email not like '%adequate%'
                  and so.coupon_code IS NULL"""
clients= pd.read_sql(clients_query, con)
clients_raw = clients

# Stocks Query

stocks_query = """select so.entity_id, soi.product_id, soi.qty_ordered, 
                    case when products.consumption_time is null then 30 else products.consumption_time end as cons_time
                    from magento.sales_order so 
                    left join magento.sales_order_item soi 
                    on so.entity_id = soi.order_id
                    left join
                    (
                    select soi.name, soi.product_id,
                    case when soi.product_id IN ('217', '248') then 180 when product_id IN ('174') then 120
                    when product_id IN ('66', '127', '227', '242', '206') then 60 
                    else 30 end as consumption_time from magento.sales_order so 
                    left join magento.sales_order_item soi 
                    on so.entity_id = soi.order_id 
                    where soi.product_type = 'simple'
                    and so.is_valid = 1
                    and so.store_id = 1
                    and so.created_at >= '2021-01-01'
                    group by 1,2) products
                    on soi.product_id = products.product_id
                    where so.is_valid = 1
                    and so.store_id = 1
                    and so.created_at >= '2021-01-01'"""
stocks = pd.read_sql(stocks_query, con)

# Phone Query

phones_query = ("select phone, email from magento.me_users mu")
phones = pd.read_sql(phones_query, con)

# Influencers

clients = clients_raw
clients['current_date'] = now 
clients['current_date'] = pd.to_datetime(clients['current_date'])
clients['influ_window_days'] = 60
clients['threshold'] = pd.to_datetime(clients.current_date) - pd.to_timedelta(pd.np.ceil(clients.influ_window_days), unit="D")
influ_exclusion = clients.drop_duplicates('customer_email')
influ_exclusion = influ_exclusion[(influ_exclusion['sales_channel'] == 'ecomm_influ') & (influ_exclusion['threshold'] < influ_exclusion['customer_max'])]
influ_exclusion = influ_exclusion.drop_duplicates('customer_email')
influ_exclusion = influ_exclusion[['customer_email']]
influ_exclusion['ex'] = 1
influ_exclusion

# 60 
vicidial_60 = vicidial
filter_list = ['CHUZNR', 'INFOSZ', 'MORE', 'AFTHRS',
'NA','ERI','DROP','DC','B','AFAX','ADC','AB','AA','A']
vicidial_60 = vicidial_60[vicidial_60.status.isin(filter_list)]
vicidial_60['current_date'] = now 
vicidial_60['current_date'] = pd.to_datetime(vicidial_60['current_date'])
vicidial_60['window_days'] = 60
vicidial_60['date_60'] = pd.to_datetime(vicidial_60.current_date) - pd.to_timedelta(pd.np.ceil(vicidial_60.window_days), unit="D")
vicidial_60 = vicidial_60[(vicidial_60['call_date'] >= vicidial_60['date_60'])]
vicidial_exclusion_60 = vicidial_60.drop_duplicates('phone_number_dialed')
vicidial_exclusion_60 = (vicidial_exclusion_60[['phone_number_dialed']])
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_60['exx'] = 1
vicidial_exclusion_60

# 150

vicidial_150 = vicidial
filter_list = ['REZBEF','REZCEN','REZINP','REZMAP','REZNST','REZYGN', 'REZWTK', 'REZCIA', 'REZKON', 'REZNIE', 'REZKNT', 'REZPWS', 'REZTZP']
vicidial_150 = vicidial_150[vicidial_150.status.isin(filter_list)]
vicidial_150['window_days'] = 150
vicidial_150['current_date'] = now 
vicidial_150['current_date'] = pd.to_datetime(vicidial_150['current_date'])
vicidial_150['date_150'] = pd.to_datetime(vicidial_150.current_date) - pd.to_timedelta(pd.np.ceil(vicidial_150.window_days), unit="D")
vicidial_150 = vicidial_150[(vicidial_150['call_date'] >= vicidial_150['date_150'])]
vicidial_exclusion_150 = vicidial_150.drop_duplicates('phone_number_dialed')
vicidial_exclusion_150 = (vicidial_150[['phone_number_dialed']])
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_150['exx'] = 1
vicidial_exclusion_150

# Wykluczenia 

nichko = vicidial
filter_list = ['DUBEL','NICHKO','ODMROZ','OSNIDO','POM',
#'PROPLA',
'ZASTAN'
#,'ODD'
]
nichko = nichko[nichko.status.isin(filter_list)]
nichko['current_date'] = now 
nichko['current_date'] = pd.to_datetime(nichko['current_date'])
nichko['window_days'] = 20000
nichko['date_20000'] = pd.to_datetime(nichko.current_date) - pd.to_timedelta(pd.np.ceil(nichko.window_days), unit="D")
nichko = nichko[(nichko['call_date'] >= nichko['date_20000'])]
vicidial_exclusion_nich = nichko.drop_duplicates('phone_number_dialed')
vicidial_exclusion_nich = (nichko[['phone_number_dialed']])
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_nich['exx'] = 1
vicidial_exclusion_nich

# Połączenie Wykluczeń

vicidial_exclusion = pd.concat([vicidial_exclusion_nich, vicidial_exclusion_60, vicidial_exclusion_150])
vicidial_exclusion = vicidial_exclusion.drop_duplicates()
vicidial_exclusion

# Order Lenght

stocks['product_length'] = stocks['cons_time'] * stocks['qty_ordered']
order_delay = stocks.groupby(['entity_id']).agg({'product_length': [np.max, np.average],  
                                                           'product_id': 'count'})

clients = clients.merge(order_delay, on='entity_id', how='left')
clients[('product_length', 'average')] = clients[('product_length', 'average')].fillna(30)
clients['avgg'] = clients[('product_length', 'average')]
clients['avgg'] = clients['avgg'].astype(int)
clients = clients[clients['max_diff'] == 0]  

clients = clients[['customer_email', 'customer_max']]



# Dataset Creation
df_main = clients.groupby(['customer_email']).agg({'customer_max': [np.max]}).reset_index()
df_main['max_date'] = df_main[('customer_max', 'amax')]
df_main = df_main.sort_values(['max_date'], ascending =[False])
df_main = df_main[['customer_email', 'max_date']]
df_main['current_date'] = now
df_main['date_diff'] = df_main['current_date'] - df_main['max_date']
df_main['date_diff'] = df_main['date_diff'].dt.days.astype('int16')
df_main['max_score'] = 10
df_main['ujemny_score'] = df_main['date_diff'] * 0.037
df_main['score'] = df_main['max_score'] - df_main['ujemny_score']
df_main = df_main[['customer_email', 'max_date', 'current_date', 'score']]
df_main

phones2 = phones
phones2['phone']=phones2['phone'].astype(str)
phones2['phone'] = phones2['phone'].str.replace(r'\+48', '', regex=True)
phones2['phone'] = phones2['phone'].str.replace(r' ', '', regex=True)
phones2.phone = pd.to_numeric(phones2.phone, errors='coerce').fillna(0).astype(np.int64)
phones2 = phones2[phones2['phone'].between(400000000, 960000000)]
phones2


# Łączenie wszystkiego

phones2 = phones2.set_index('email')
df_main2 = df_main.set_index('customer_email')
df_main2 = df_main.merge(phones2, left_on='customer_email', right_on='email', how='left')
df_main2 = df_main2[df_main2['phone'].notna()]
df_main2 = df_main2.set_index('phone')
df_main2 = df_main2.merge(score_pol, left_index=True, right_index=True, how='left')
vicidial_exclusion = vicidial_exclusion.set_index('phone_number_dialed')
df_main2 = df_main2.merge(vicidial_exclusion, left_index=True, right_index=True, how='left')
df_main2 = df_main2.reset_index()
df_main2 = df_main2.merge(influ_exclusion, left_on=[('customer_email', '')], right_on='customer_email', how='left')

# Sumowanie scoru

df_main2['stat_sum_ost'] = df_main2['stat_sum_ost'].fillna(0)
df_main2['score_sum'] = df_main2[('score', '')] + df_main2['stat_sum_ost']

# Wyrzucenie exx

df_final = df_main2[(df_main2['ex'].isnull()) & (df_main2['exx'].isnull())]

# Sort po score i nazywanie kolumn

df_final = df_final.sort_values(by='score_sum', ascending=False)
df_final = df_final.rename(columns={('score', ''): 'Score_czas', 'stat_sum_ost': 'Score_historia', 'score_sum': 'Score_suma', 'index': 'phone'})
df_final = df_final[['phone', 'Score_czas', 'Score_historia', 'Score_suma']]

# Import wszystkich plików z folderu

path = r'C:\Users\stefa\Desktop\csvki\model_cc'  
all_files = glob.glob(os.path.join(path, "*.csv"))     
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
concatenated_df = concatenated_df.rename(columns={'phone':'phone2'})
concatenated_df

# Dołączenie go do poprzedniego, finałowego datasetu

df_final2 = df_final.merge(concatenated_df, left_on='phone', right_on='phone2', how='left')

# Wyrzucenie duplikatów, czyli numerów, które były w poprzednich datasetach

df_final2 = df_final2[df_final2['phone2'].isnull()]
df_final2 = df_final2[['phone', 'Score_czas', 'Score_historia', 'Score_suma']]
df_final2




Script is starting, the current time is 2022-06-20 16:40:29.983890


C:\Users\stefa\AppData\Local\Temp/ipykernel_25492/250372259.py:170: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  clients['threshold'] = pd.to_datetime(clients.current_date) - pd.to_timedelta(pd.np.ceil(clients.influ_window_days), unit="D")
C:\Users\stefa\AppData\Local\Temp/ipykernel_25492/250372259.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vicidial_60['current_date'] = now
C:\Users\stefa\AppData\Local\Temp/ipykernel_25492/250372259.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

C:\Users\stefa\Anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(
C:\Users\stefa\Anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


,phone,Score_czas,Score_historia,Score_suma
0,728461002.0,8.816,80.0,88.816
2,512356070.0,8.816,54.0,62.816
3,883273570.0,9.741,52.0,61.741
13,692012125.0,6.929,42.0,48.929
14,691414333.0,6.892,42.0,48.892
...,...,...,...,...
109658,664369819.0,-6.021,-92.0,-98.021
109660,570690880.0,-7.057,-98.0,-105.057
109661,511138958.0,-2.765,-116.0,-118.765
109662,697830891.0,-4.430,-128.0,-132.430


In [36]:

# Baza 3000 osób 

df_final_3000 = df_final2[['phone']]
df_final_3000= df_final_3000.astype(int)
df_final_3000 = df_final_3000.head(3000)
df_final_3000.to_csv('df_final_3000.csv')

,phone
0,728461002.0
2,512356070.0
3,883273570.0
13,692012125.0
14,691414333.0
...,...
109658,664369819.0
109660,570690880.0
109661,511138958.0
109662,697830891.0


In [50]:
df_final2.head(100)

,phone,Score_czas,Score_historia,Score_suma
0,728461002.0,8.816,80.0,88.816
2,512356070.0,8.816,54.0,62.816
3,883273570.0,9.741,52.0,61.741
13,692012125.0,6.929,42.0,48.929
14,691414333.0,6.892,42.0,48.892
19,669206159.0,7.780,40.0,47.780
20,669058516.0,9.741,38.0,47.741
25,506693541.0,8.964,36.0,44.964
26,508063155.0,8.964,36.0,44.964
28,507130056.0,6.707,38.0,44.707
